In [25]:
# Imports
import requests
import pandas as pd

In [26]:
# Official FPL API Endpoints
API_URL = "https://fantasy.premierleague.com/api/"
ALL_DATA_ENDPOINT = "bootstrap-static"
FIXTURES_ENDPOINT = "fixtures"
PLAYER_DATA_ENDPOINT = "element-summary"

In [27]:
# Fetch all the data
data = requests.get(API_URL + ALL_DATA_ENDPOINT).json()

In [28]:
# Check the different data types in the result
data.keys()

dict_keys(['events', 'game_settings', 'phases', 'teams', 'total_players', 'elements', 'element_stats', 'element_types'])

In [29]:
# Create a data frame for all the players. This has overall stats for the players so far this season
players_df = pd.DataFrame(data["elements"])

In [30]:
players_df.head()

,chance_of_playing_next_round,chance_of_playing_this_round,code,cost_change_event,cost_change_event_fall,cost_change_start,cost_change_start_fall,dreamteam_count,element_type,ep_next,...,now_cost_rank,now_cost_rank_type,form_rank,form_rank_type,points_per_game_rank,points_per_game_rank_type,selected_rank,selected_rank_type,starts_per_90,clean_sheets_per_90
0,0.0,0.0,232223,0,0,-1,1,0,4,0.0,...,525,94,563,54,625,65,319,49,0.00,0.00
1,NaN,NaN,58822,0,0,-1,1,0,2,0.5,...,723,228,399,134,491,166,253,101,0.00,0.00
2,100.0,100.0,153256,0,0,-1,1,0,3,0.5,...,494,286,544,195,53,30,419,138,0.00,0.00
3,0.0,0.0,438098,0,0,-1,1,0,3,0.0,...,128,81,758,328,198,89,426,142,0.76,0.38
4,100.0,100.0,226597,0,0,-1,1,0,2,3.1,...,250,32,112,31,156,50,22,6,0.98,0.33


In [31]:
# Get detailed info for each player from players_df.
player_history_df = pd.DataFrame()
for i in range(len(players_df)):
    # Fetch the player id
    player_id = players_df.iloc[i]["id"]
    
    # Fetch player status so that transferred and loaned players can be excluded
    player_status = players_df.iloc[i]["status"]
    
    if player_status == "n" or player_status == "u":
        continue
    
    # Get the players' name
    first_name = players_df.iloc[i]["first_name"]
    last_name = players_df.iloc[i]["second_name"]
    
    # Get the players' position
    position = players_df.iloc[i]["element_type"]
    
    # Get the individual player data
    player_data = requests.get(API_URL + PLAYER_DATA_ENDPOINT + f"/{player_id}").json()
    
    # Extract past seasons' data
    player_history = pd.DataFrame(player_data["history_past"])
    
    # Add player id and name to the data frame
    player_history["player_id"] = player_id
    player_history["first_name"] = first_name
    player_history["last_name"] = last_name
    player_history["position"] = position
    
    player_history_df = pd.concat([player_history_df, player_history])

    print(f"Fetching data for {first_name} {last_name}")
    if i % 20 == 0:
        print(f"Progress: {(i / len(players_df) * 100): .2f}%")



Fetching data for Cédric Alves Soares
Fetching data for Mohamed Elneny
Fetching data for Fábio Ferreira Vieira
Fetching data for Gabriel dos Santos Magalhães
Fetching data for Kai Havertz
Fetching data for Gabriel Fernando de Jesus
Fetching data for Jorge Luiz Frello Filho
Fetching data for Jakub Kiwior
Fetching data for Gabriel Martinelli Silva
Fetching data for Eddie Nketiah
Fetching data for Martin Ødegaard
Fetching data for Thomas Partey
Fetching data for Aaron Ramsdale
Fetching data for Bukayo Saka
Fetching data for William Saliba
Fetching data for Emile Smith Rowe
Fetching data for Takehiro Tomiyasu
Fetching data for Leandro Trossard
Fetching data for Benjamin White
Fetching data for Oleksandr Zinchenko
Fetching data for David Raya Martin
Fetching data for Declan Rice
Fetching data for Reiss Nelson
Fetching data for Jurriën Timber
Fetching data for Karl Hein
Fetching data for Charles Sagoe
Fetching data for Reuell Walters
Fetching data for Bradley Ibrahim
Fetching data for Alexan

In [32]:
player_history_df.head()

,season_name,element_code,start_cost,end_cost,total_points,minutes,goals_scored,assists,clean_sheets,goals_conceded,...,ict_index,starts,expected_goals,expected_assists,expected_goal_involvements,expected_goals_conceded,player_id,first_name,last_name,position
0,2015/16,58822.0,50.0,47.0,86.0,1965.0,0.0,2.0,9.0,19.0,...,0.0,0.0,0.00,0.00,0.00,0.00,2,Cédric,Alves Soares,2
1,2016/17,58822.0,50.0,49.0,102.0,2515.0,0.0,3.0,11.0,36.0,...,139.6,0.0,0.00,0.00,0.00,0.00,2,Cédric,Alves Soares,2
2,2017/18,58822.0,50.0,47.0,85.0,2794.0,0.0,3.0,7.0,44.0,...,112.3,0.0,0.00,0.00,0.00,0.00,2,Cédric,Alves Soares,2
3,2018/19,58822.0,45.0,42.0,52.0,1493.0,1.0,2.0,4.0,31.0,...,63.9,0.0,0.00,0.00,0.00,0.00,2,Cédric,Alves Soares,2
4,2019/20,58822.0,50.0,48.0,61.0,1553.0,1.0,1.0,4.0,20.0,...,68.7,0.0,0.00,0.00,0.00,0.00,2,Cédric,Alves Soares,2


In [33]:
# Save all the dataframes
player_history_df.to_csv("data/player_history.csv", index=None)